# Couchbase Tutorial: Create, Get, Update, Delete

## In this tutorial, we'll cover:
* Installing the Couchbase Python SDK module
* Connecting to a Couchbase Server
* Creating documents with UUID keys and JSON values
* Fetching a document by its key
* Updating a document
* Deleting documents
* Listing documents in a collection, using SQL++ query

## Install Packages 

In [1]:
%pip install --upgrade --quiet couchbase

Note: you may need to restart the kernel to use updated packages.


## Set Environment Variables

In [2]:
import env
env.load()

## Module Import And Client Setup

In [3]:
import os
import uuid
from datetime import timedelta
import couchbase.subdocument as SD
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.options import (ClusterOptions, ClusterTimeoutOptions, QueryOptions, MutateInOptions)
from couchbase.exceptions import ScopeAlreadyExistsException, CollectionAlreadyExistsException
from couchbase.management.collections import CreateCollectionSettings

COUCHBASE_USERNAME = os.environ['COUCHBASE_USERNAME']
COUCHBASE_PASSWORD = os.environ['COUCHBASE_PASSWORD']
COUCHBASE_BUCKET_NAME = os.environ['COUCHBASE_BUCKET_NAME']
COUCHBASE_URL = os.environ['COUCHBASE_URL']

auth = PasswordAuthenticator(
    COUCHBASE_USERNAME,
    COUCHBASE_PASSWORD
)
cluster = Cluster(COUCHBASE_URL, ClusterOptions(auth))
cluster.wait_until_ready(timedelta(seconds=5))
bucket = cluster.bucket(COUCHBASE_BUCKET_NAME)

scope_name = "cillers_play"
collection_name_products = "products"

## Prepare The Data Structure

In [4]:
import data_structure_couchbase

data_structure_spec = {
    scope_name: [
        collection_name_products
    ]
}

data_structure_couchbase.create(bucket, data_structure_spec)

Scope 'cillers_play' created successfully.
Collection 'products' created successfully in scope 'cillers_play'.


## Create A Document

There are two different methods for creating a document in Couchbase: insert and upsert. Insert fails if there already exists a document with the same key. Upsert creates a document if no document exists with the same key and completely replaces any preexisting document with the same key. Here we use insert because the key is random with practically zero chance of colliding with an existing key. But, often keys are based on emails or other non-random data, because that enables uniqueness constraint and fast document fetching through direct key access instead of having to rely on SQL++ queries. 

In [5]:
key = str(uuid.uuid1())
collection = bucket.scope(scope_name).collection(collection_name_products)
result = collection.insert(key, {"name": "Jupyter A"})
print(result)

MutationResult:result:{value={'cas': 1731961307449655296, 'key': 'bc74e526-a5ea-11ef-bceb-0242c0a87517', 'mutation_token': <pycbc_core.mutation_token object at 0xffff99332490>}}


## Update A Document

There are three different methods to update a document in Couchbase: a) replacing the entire document using `replace`, which will fail if no document exists with the specified key b) replacing the document using `upsert`, which will create a new document if no document existed with the specified key, and c) updating specific values in the document using `mutate_in`. 

In [6]:
# First, let's add a document to modify
key = str(uuid.uuid1())
collection = bucket.scope(scope_name).collection(collection_name_products)
result = collection.upsert(key, {"name": "Jupyter A", "type": "notebook"})
print(f"Document created: {result}\n")

# Now, let's modify the document using replace() - This replaces the entire document
try:
    result = collection.replace(key, {"name": "Jupyter B", "type": "notebook", "updated": True, "tags":["tag1"]})
    print(f"Document replaced: {result}\n")
except Exception as e:
    print(f"Error replacing document: {e}\n")

# Now, let's modify the document using mutate_in() - This updates specific fields in the document
try:
    result = collection.mutate_in(key, [
        SD.upsert("color", "blue"),
        SD.remove("updated"),
        SD.array_append("tags", "modified")
    ])
    print(f"Document updated: {result}\n")
except Exception as e:
    print(f"Error updating document: {e}\n")

# 3. Using get() and replace() - This allows you to modify the document based on its current state
try:
    doc = collection.get(key)
    content = doc.value
    content["version"] = 2
    result = collection.replace(key, content)
    print(f"Document modified and upserted: {result}\n")
except Exception as e:
    print(f"Error modifying and upserting document: {e}\n")

# Print the final document
try:
    doc = collection.get(key)
    print(f"Final document: {doc.value}\n")
except Exception as e:
    print(f"Error retrieving document: {e}\n")

Document created: MutationResult:result:{value={'cas': 1731961311531237376, 'key': 'bee3890c-a5ea-11ef-bceb-0242c0a87517', 'mutation_token': <pycbc_core.mutation_token object at 0xffff9753de30>}}

Document replaced: MutationResult:result:{value={'cas': 1731961311532613632, 'key': 'bee3890c-a5ea-11ef-bceb-0242c0a87517', 'mutation_token': <pycbc_core.mutation_token object at 0xffff9753ded0>}}

Document updated: MutateInResult:result:{value={'cas': 1731961311534317568, 'flags': None, 'key': 'bee3890c-a5ea-11ef-bceb-0242c0a87517', 'mutation_token': <pycbc_core.mutation_token object at 0xffff9753df70>, 'value': [{'opcode': 200, 'status': 0, 'path': 'color', 'original_index': 0}, {'opcode': 201, 'status': 0, 'path': 'updated', 'original_index': 1}, {'opcode': 203, 'status': 0, 'path': 'tags', 'original_index': 2}]}}

Document modified and upserted: MutationResult:result:{value={'cas': 1731961311535366144, 'key': 'bee3890c-a5ea-11ef-bceb-0242c0a87517', 'mutation_token': <pycbc_core.mutation_t

## Get The Created Document

In [7]:
get_result = collection.get(key)
print(get_result.content_as[dict])

{'name': 'Jupyter B', 'type': 'notebook', 'tags': ['tag1', 'modified'], 'color': 'blue', 'version': 2}


## Delete The Created Document

In [8]:
remove_result = collection.remove(key)
print("Remove Result:", remove_result)

Remove Result: MutationResult:result:{value={'cas': 1731961346167144448, 'key': 'bee3890c-a5ea-11ef-bceb-0242c0a87517', 'mutation_token': <pycbc_core.mutation_token object at 0xffff9753de70>}}
